In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/datasetFoodImage/images.zip" -d "/content/sample_data/data"

In [7]:
import json
import os
import tensorflow as tf
from google.colab import files
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras import Sequential, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from tensorflow_hub import KerasLayer
from tensorflow.keras.metrics import Precision,Recall
import math

In [4]:
EfficientV2LPath = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_l/feature_vector/2'

In [6]:
modelCNN = KerasLayer(EfficientV2LPath , trainable = False, input_shape = (250,250,3), name = 'EfficientNet')

In [8]:
def build_model(modelCNN):
  model = Sequential([
    Input(shape=(250,250,3)),
    modelCNN,   
    Dense(108, activation="softmax")
  ])
  model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["accuracy", Precision(), Recall()])
  return model

In [39]:
model = build_model(modelCNN)

In [40]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 EfficientNet (KerasLayer)   (None, 1280)              117746848 
                                                                 
 dense_4 (Dense)             (None, 108)               138348    
                                                                 
Total params: 117,885,196
Trainable params: 138,348
Non-trainable params: 117,746,848
_________________________________________________________________


In [13]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.1)

In [14]:
train_data_generator = datagen.flow_from_directory(
    "/content/sample_data/data/images",
    target_size=(250,250),
    subset='training'
)

Found 91064 images belonging to 108 classes.


In [15]:
valid_data_generator = datagen.flow_from_directory(
    "/content/sample_data/data/images",
    target_size=(250,250),
    subset='validation'
)

Found 10115 images belonging to 108 classes.


In [16]:
freq = 91064/32

In [36]:
checkpoint = ModelCheckpoint("/content/sample_data/model/Effnet_{epoch:02d}_{accuracy:.2f}.h5", verbose=1, mode='auto', save_freq=2846)

In [37]:
class myCallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    path = '/content/sample_data/trainingHistoryEfficientNet.json'
    with open(path) as f:
      data = json.load(f)
    os.remove(path)
    dictLogs = {
        'loss': logs.get('loss'),
        'accuracy': logs.get('accuracy'),
        'val_loss': logs.get('val_loss'),
        'val_accuracy': logs.get('val_accuracy'),
        'val_precision': logs.get('val_precision_4'),
        'val_recall': logs.get('val_recall_4'),
        'recall': logs.get('recall_4'),
        'precision': logs.get('precision_4')
    }
    data.append(dictLogs)
    json_object = json.dumps(data, indent = 4)
    with open(path, "w") as outJson:
      outJson.write(json_object)

In [38]:
callbacks = myCallback()

In [41]:
model.fit(train_data_generator, validation_data=valid_data_generator, epochs=5,callbacks=[checkpoint, callbacks])

Epoch 1/5
2845/2846 [============================>.] - ETA: 1s - loss: 1.0524 - accuracy: 0.7335 - precision_4: 0.8811 - recall_4: 0.6279
Epoch 1: saving model to /content/sample_data/model/Effnet_01_0.73.h5
2846/2846 [==============================] - 3265s 1s/step - loss: 1.0522 - accuracy: 0.7336 - precision_4: 0.8812 - recall_4: 0.6280 - val_loss: 0.8125 - val_accuracy: 0.7901 - val_precision_4: 0.8805 - val_recall_4: 0.7339
Epoch 2/5
2845/2846 [============================>.] - ETA: 1s - loss: 0.6504 - accuracy: 0.8230 - precision_4: 0.9003 - recall_4: 0.7679
Epoch 2: saving model to /content/sample_data/model/Effnet_02_0.82.h5
2846/2846 [==============================] - 3279s 1s/step - loss: 0.6505 - accuracy: 0.8230 - precision_4: 0.9003 - recall_4: 0.7679 - val_loss: 0.8170 - val_accuracy: 0.7914 - val_precision_4: 0.8675 - val_recall_4: 0.7462
Epoch 3/5
2845/2846 [============================>.] - ETA: 1s - loss: 0.5318 - accuracy: 0.8530 - precision_4: 0.9161 - recall_4: 0.8